**Zeolite Catalysts: A Numerical Analysis Approach for Raman**

Aldo Bushati

CME 504 - Prof Nav Nidhi Rajput 

SBU ID: 112880257

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.integrate import simps
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score


def plot_conversion_from_text(temperature_array, conversion_file, xlabel='Temperature (°C)', ylabel='CO Conversion (%)', title='ZSM-5 Catalyst Activity'):
    """
    Plots CO conversion data against temperature from a text file.

    Parameters:
        temperature_array (array-like): Array containing temperature data.
        conversion_file (str): Path to the text file containing conversion data.
        xlabel (str, optional): Label for the x-axis. Defaults to 'Temperature (°C)'.
        ylabel (str, optional): Label for the y-axis. Defaults to 'CO Conversion (%)'.
        title (str, optional): Title of the plot. Defaults to 'ZSM-5 Catalyst Activity'.

    Returns:
        plotly.graph_objs.Figure: A Plotly figure object displaying the CO conversion data.
    """
    # Read conversion data from text file
    with open(conversion_file, 'r') as file:
        lines = file.readlines()
        temperatures = []
        conversions = []
        for line in lines:
            parts = line.strip().split('|')
            if len(parts) == 2:
                try:
                    temperature = float(parts[0].strip())
                    conversion = float(parts[1].strip())
                    temperatures.append(temperature)
                    conversions.append(conversion)
                except ValueError:
                    pass

    # Create a figure
    fig = go.Figure()

    # Plot conversion data
    fig.add_trace(go.Scatter(x=temperatures, y=conversions, mode='lines+markers', name='Conversion', line=dict(color='blue')))

    # Highlight maximum conversion point
    if conversions:  # Check if conversions list is not empty
        max_conversion_index = np.argmax(conversions)
        max_conversion_temp = temperatures[max_conversion_index]
        max_conversion_value = conversions[max_conversion_index]
        fig.add_trace(go.Scatter(x=[max_conversion_temp], y=[max_conversion_value], mode='markers', name=f'Max Conversion ({max_conversion_temp}°C)', marker=dict(color='red', size=10)))

        # Add annotations
        annotations = [
            dict(
                x=max_conversion_temp,
                y=max_conversion_value,
                xref="x",
                yref="y",
                text=f'Max Conversion: {max_conversion_value} (°C: {max_conversion_temp})',
                showarrow=True,
                arrowhead=7,
                ax=0,
                ay=-40
            )
        ]
        fig.update_layout(annotations=annotations)

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            title=xlabel,
            tickfont=dict(size=14)  # Adjust font size for x-axis labels
        ),
        yaxis=dict(
            title=ylabel,
            tickfont=dict(size=14)  # Adjust font size for y-axis labels
        ),
        hovermode='closest',
        showlegend=True,
        template='plotly_white'  # Makes the graph with a white background
    )

    return fig

# Example usage for ZSM-5 catalyst activity
temperature_array = np.arange(0, 401, 4)  # Adjust temperature array accordingly
conversion_file = "zsm5_conversion_data.txt"
conversion_plot = plot_conversion_from_text(temperature_array, conversion_file)

# Save the interactive plot to an HTML file
pyo.plot(conversion_plot, filename='zsm5_catalyst_activity.html')



'zsm5_catalyst_activity.html'

In [2]:
def plot_raman_from_text(text_files, colors=None, titles=None, peak_threshold=0.1, prominence_threshold=0.1, spacing=200):
    """
    Plots Raman spectra from text files.

    Parameters:
        text_files (str or list of str): Path or list of paths to the text file(s) containing Raman spectra data.
        colors (list of str, optional): List of colors to be used for each spectrum. Defaults to None.
        titles (list of str, optional): List of titles for each spectrum. Defaults to None.
        peak_threshold (float, optional): Threshold for peak detection as a fraction of the maximum intensity. Defaults to 0.1.
        prominence_threshold (float, optional): Threshold for peak prominence. Defaults to 0.1.
        spacing (int, optional): Spacing between spectra on the plot. Defaults to 200.

    Returns:
        plotly.graph_objs.Figure: A Plotly figure object displaying the Raman spectra.
    """
    if not isinstance(text_files, list):
        text_files = [text_files]

    if colors is None:
        colors = ['blue', 'green', 'cyan', 'orange', 'purple']  # Default colors

    if titles is None:
        titles = [f'Raman Spectrum {i+1}' for i in range(len(text_files))]  # Default titles

    fig = go.Figure()

    for i, text_file in enumerate(text_files):
        # Read the text file and extract wavenumbers and intensities
        with open(text_file, 'r') as file:
            lines = file.readlines()
            x = []
            y = []
            for line in lines:
                parts = line.strip().split()  # Assuming wavenumber and intensity are separated by whitespace
                if len(parts) == 2:
                    x.append(float(parts[0]))
                    y.append(float(parts[1]))

        # Interpolate the data using linear interpolation
        f = interp1d(x, y, kind='linear')

        # Generate new x values for interpolation
        x_smooth = np.linspace(min(x), max(x), 1000)
        y_smooth = f(x_smooth)

        # Calculate the offset for this spectrum
        offset = i * spacing

        # Plot the Raman spectrum with the offset
        fig.add_trace(go.Scatter(x=x_smooth, y=y_smooth + offset, mode='lines', name=titles[i], line=dict(color=colors[i % len(colors)])))

        # Peak detection and annotation
        peaks, properties = find_peaks(y_smooth, height=peak_threshold * max(y_smooth), prominence=prominence_threshold)

        # Scatter plot for peaks
        fig.add_trace(go.Scatter(x=np.array(x_smooth)[peaks], y=np.array(y_smooth)[peaks] + offset, mode='markers', marker=dict(color='red'), showlegend=False))

        # Calculate intensity of disordered and graphitic bands
        disordered_band_intensity = max(y_smooth[np.logical_and(x_smooth >= 1450, x_smooth <= 1500)])
        graphitic_band_intensity = max(y_smooth[np.logical_and(x_smooth >= 1600, x_smooth <= 1650)])

        # Add intensity ratio to legend text
        intensity_ratio = disordered_band_intensity / graphitic_band_intensity
        title_with_ratio = f'{titles[i]} (Intensity Ratio (I_D/I_G): {intensity_ratio:.2f})'
        fig.update_traces(name=title_with_ratio, selector=dict(name=titles[i]))

    # Add annotation for disordered band
    fig.add_annotation(x=1450, y=max(y_smooth) - 200, text="Disordered Band (I_D)", showarrow=True, arrowhead=1, ax=0, ay=-40)

    # Add annotation for graphitic band
    fig.add_annotation(x=1610, y=max(y_smooth) + 0, text="Graphitic Band (I_G)", showarrow=True, arrowhead=1, ax=0, ay=-40)

    fig.update_layout(
        title='Raman Spectra',
        xaxis=dict(
            title='Wavenumber',
            tickfont=dict(size=14)  # Adjust font size for x-axis labels
        ),
        yaxis=dict(
            title='Intensity',
            tickfont=dict(size=14)  # Adjust font size for y-axis labels
        ),
        hovermode='closest',
        showlegend=True
    )

    return fig

# Example usage
text_files = ["raman_spectrum1.txt", "raman_spectrum2.txt", "raman_spectrum3.txt"]
raman_plot = plot_raman_from_text(text_files, titles=["Mo-ZSM-5_MoO2", "Mo-ZSM-5_BM-Mo Metal", "Mo-ZSM-5_BM-MoO3-bulk"], prominence_threshold=15, spacing=0)

# Save the interactive plot to an HTML file
pyo.plot(raman_plot, filename='raman_spectra.html')

'raman_spectra.html'

In [3]:
def max_intensity_normalize_raman(y):
    """
    Normalize Raman spectra by dividing by the maximum intensity.

    Parameters:
    y (array-like): Array containing the intensities.

    Returns:
    array-like: Normalized intensities.
    """
    max_intensity = max(y)
    return np.array(y) / max_intensity

def plot_normalized_raman_from_text(text_files, normalization_method='area', colors=None, titles=None, peak_threshold=0.1, prominence_threshold=0.1, spacing=200):
    """
    Plots normalized Raman spectra from text files.

    Parameters:
        text_files (str or list of str): Path or list of paths to the text file(s) containing Raman spectra data.
        normalization_method (str, optional): Method for normalization. 'area' normalizes by the area under the curve, 'max_intensity' normalizes by the maximum intensity. Defaults to 'area'.
        colors (list of str, optional): List of colors to be used for each spectrum. Defaults to None.
        titles (list of str, optional): List of titles for each spectrum. Defaults to None.
        peak_threshold (float, optional): Threshold for peak detection as a fraction of the maximum intensity. Defaults to 0.1.
        prominence_threshold (float, optional): Threshold for peak prominence. Defaults to 0.1.
        spacing (int, optional): Spacing between spectra on the plot. Defaults to 200.

    Returns:
        plotly.graph_objs.Figure: A Plotly figure object displaying the normalized Raman spectra.
    """
    if not isinstance(text_files, list):
        text_files = [text_files]

    if colors is None:
        colors = ['blue', 'green', 'cyan', 'orange', 'purple']  # Default colors

    if titles is None:
        titles = [f'Normalized Raman Spectrum {i+1}' for i in range(len(text_files))]  # Default titles

    fig = go.Figure()

    max_intensity = 0  # Track maximum intensity for adjusting y-axis range

    for i, text_file in enumerate(text_files):
        # Read the text file and extract wavenumbers and intensities
        with open(text_file, 'r') as file:
            lines = file.readlines()
            x = []
            y = []
            for line in lines:
                parts = line.strip().split()  # Assuming wavenumber and intensity are separated by whitespace
                if len(parts) == 2:
                    x.append(float(parts[0]))
                    y.append(float(parts[1]))

        # Perform normalization based on selected method
        if normalization_method == 'area':
            x, y = area_normalize_raman(x, y)
        elif normalization_method == 'max_intensity':
            y = max_intensity_normalize_raman(y)
        else:
            raise ValueError("Invalid normalization method. Supported methods: 'area', 'max_intensity'")

        # Update max_intensity
        max_intensity = max(max_intensity, max(y))

        # Calculate the area under the curve
        area = abs(np.trapz(y, x))

        # Plot the normalized Raman spectrum with the offset
        fig.add_trace(go.Scatter(x=x, y=y + i * spacing, mode='lines', name=f'{titles[i]} (Area: {area:.2f})', line=dict(color=colors[i % len(colors)])))

        # Peak detection and annotation
        peaks, properties = find_peaks(y, height=peak_threshold * max(y), prominence=prominence_threshold)

        # Scatter plot for peaks
        fig.add_trace(go.Scatter(x=np.array(x)[peaks], y=np.array(y)[peaks] + i * spacing, mode='markers', marker=dict(color='red'), showlegend=False))

    fig.update_layout(
        title='Normalized Raman Spectra',
        xaxis=dict(
            title='Wavenumber',
            tickfont=dict(size=14)  # Adjust font size for x-axis labels
        ),
        yaxis=dict(
            title='Intensity (Normalized)',
            tickfont=dict(size=14),  # Adjust font size for y-axis labels
            range=[0, max_intensity * 1.1]  # Set y-axis range to accommodate normalized intensities
        ),
        hovermode='closest',
        showlegend=True
    )

    return fig

# Example usage with normalized Raman spectra using max intensity normalization
normalized_raman_plot = plot_normalized_raman_from_text(text_files, normalization_method='max_intensity', titles=["Mo-ZSM-5_MoO2", "Mo-ZSM-5_BM-Mo Metal", "Mo-ZSM-5_BM-MoO3-bulk"], prominence_threshold=15, spacing=0)

# Save the interactive plots to HTML files
pyo.plot(normalized_raman_plot, filename='normalized_raman_spectra_max_intensity.html')

'normalized_raman_spectra_max_intensity.html'

In [4]:
def gaussian(x, a, mu, sigma):
    return a * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

def lorentzian(x, a, mu, gamma):
    return a / (1 + ((x - mu) / gamma) ** 2)

def fit_peak(x, y, peak_function):
    """
    Fits a peak function to the given data.

    Parameters:
        x (array-like): Array containing the x-values.
        y (array-like): Array containing the y-values.
        peak_function (callable): The peak function to fit (e.g., Gaussian or Lorentzian).

    Returns:
        tuple: A tuple containing the parameters of the fitted peak function, the fitted y-values, and the coefficient of determination (R^2).
    """
    p0 = [max(y), x[np.argmax(y)], 10]  # Initial guess for parameters
    params, cov = curve_fit(peak_function, x, y, p0=p0)
    y_fit = peak_function(x, *params)
    residuals = y - y_fit
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    return params, y_fit, r_squared

def area_normalize_raman(x, y):
    """
    Normalize Raman spectra by dividing by the area under the curve.

    Parameters:
    x (array-like): Array containing the wavenumbers.
    y (array-like): Array containing the intensities.

    Returns:
    array-like: Normalized intensities.
    """
    area = simps(y, x)
    return x, y / area

def plot_normalized_raman_peak_range_from_text(text_files, titles=None, prominence_threshold=0.1, spacing=0):
    """
    Plots normalized Raman spectra within the specified wavenumber range, along with Gaussian and Lorentzian fits.

    Parameters:
        text_files (str or list): Path or list of paths to the text files containing Raman spectra data.
        titles (list, optional): Titles for the Raman spectra plots. Default is None.
        prominence_threshold (float, optional): Minimum prominence of peaks for peak detection. Default is 0.1.
        spacing (int, optional): Spacing between plotted spectra. Default is 0.

    Returns:
        plotly.graph_objs._figure.Figure: Plotly figure object containing the plotted spectra.
    """
    if not isinstance(text_files, list):
        text_files = [text_files]

    if titles is None:
        titles = [f'Normalized Raman Spectrum {i+1}' for i in range(len(text_files))]  # Default titles

    fig = go.Figure()

    # Lists to store data for averaging
    all_gaussian_fits = []
    all_lorentzian_fits = []

    for i, text_file in enumerate(text_files):
        # Read the text file and extract wavenumbers and intensities
        with open(text_file, 'r') as file:
            lines = file.readlines()
            x = []
            y = []
            for line in lines:
                parts = line.strip().split()  # Assuming wavenumber and intensity are separated by whitespace
                if len(parts) == 2:
                    x.append(float(parts[0]))
                    y.append(float(parts[1]))

        # Perform normalization
        x, y = area_normalize_raman(x, y)

        # Convert to numpy arrays for easier manipulation
        x = np.array(x)
        y = np.array(y)

        # Find indices within the desired range (1500-1700 cm^-1)
        indices = np.where((x >= 1500) & (x <= 1700))[0]

        # Extract the data points within the range
        x_range = x[indices]
        y_range = y[indices]

        if len(y_range) == 0:  # Check if the intensity array is empty
            print(f"No data points found in the range for {titles[i]}. Skipping.")
            continue  # Skip this spectrum if there are no intensity values in the specified range

        # Plot the Raman spectrum with the offset
        fig.add_trace(go.Scatter(x=x_range, y=y_range + i * spacing, mode='lines', name=titles[i]))

        # Fit Gaussian peak
        params_gaussian, y_fit_gaussian, r2_gaussian = fit_peak(x_range, y_range, gaussian)

        # Fit Lorentzian peak
        params_lorentzian, y_fit_lorentzian, r2_lorentzian = fit_peak(x_range, y_range, lorentzian)

        # Store the fits for averaging
        all_gaussian_fits.append(y_fit_gaussian)
        all_lorentzian_fits.append(y_fit_lorentzian)

    # Calculate average Gaussian and Lorentzian fits
    avg_gaussian_fit = np.mean(all_gaussian_fits, axis=0)
    avg_lorentzian_fit = np.mean(all_lorentzian_fits, axis=0)

    # Add average Gaussian fit line
    fig.add_trace(go.Scatter(x=x_range, y=avg_gaussian_fit, mode='lines', line=dict(color='black', dash='dash'), name='Average Gaussian Fit'))

    # Add average Lorentzian fit line
    fig.add_trace(go.Scatter(x=x_range, y=avg_lorentzian_fit, mode='lines', line=dict(color='black', dash='dot'), name='Average Lorentzian Fit'))

    fig.update_layout(
        title='Normalized Raman Spectra with Gaussian and Lorentzian Fit (1500-1700 cm^-1)',
        xaxis=dict(
            title='Wavenumber (cm^-1)',
            tickfont=dict(size=14),  # Adjust font size for x-axis labels
            range=[1500, 1700]  # Set the x-axis range to the specified wavenumber range
        ),
        yaxis=dict(
            title='Intensity (Normalized)',
            tickfont=dict(size=14),  # Adjust font size for y-axis labels
        ),
        hovermode='closest',
        showlegend=True
    )

    # Add legend label annotations
    fig.add_annotation(x=0.5, y=-0.15, text="Lines:", showarrow=False, xref="paper", yref="paper", xanchor="center", yanchor="top", font=dict(size=12, color="black"))

    return fig

# Example usage with multiple spectra
text_files = ["raman_spectrum1.txt", "raman_spectrum2.txt", "raman_spectrum3.txt"]
normalized_raman_peak_plot = plot_normalized_raman_peak_range_from_text(text_files, titles=["Mo-ZSM-5_MoO2", "Mo-ZSM-5_BM-Mo Metal", "Mo-ZSM-5_BM-MoO3-bulk"], prominence_threshold=0, spacing=0)

# Save the interactive plots to HTML files
pyo.plot(normalized_raman_peak_plot, filename='normalized_raman_spectra_1500_1700cm^-1_range.html')

'normalized_raman_spectra_1500_1700cm^-1_range.html'